In [2]:
# Imports
import torch
import clip
import faiss
import os
from PIL import Image
from langchain.vectorstores import FAISS as LangFAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from faster_whisper import WhisperModel

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Whisper (audio to text)
def transcribe_audio(file_path):
    print("🎤 Transcribing audio with faster-whisper...")
    model = WhisperModel("base", device=device)
    segments, _ = model.transcribe(file_path)
    transcription = " ".join([segment.text for segment in segments])
    return transcription

# 2. CLIP (image to embedding)
def embed_image(file_path, model, preprocess):
    print("🖼️ Embedding image using CLIP...")
    image = preprocess(Image.open(file_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features.cpu().numpy()

# 3. Text embedding
def embed_text_chunks(docs):
    print("📚 Embedding text chunks...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = LangFAISS.from_documents(docs, embeddings)
    return vectorstore

# 4. Split large text corpus
def split_documents(texts):
    docs = [Document(page_content=t) for t in texts]
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(docs)

# 5. Load LLM (Light model for Colab/PC)
def load_llm():
    print("🧠 Loading FLAN-T5-Base...")
    model_id = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
    return HuggingFacePipeline(pipeline=pipe)

# 6. Load CLIP
def init_clip():
    print("⚙️ Loading CLIP model...")
    model, preprocess = clip.load("ViT-B/32", device=device)
    return model, preprocess

# 7. Create QA chain
def create_multimodal_qa(corpus):
    chunks = split_documents(corpus)
    vectorstore = embed_text_chunks(chunks)
    llm = load_llm()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# 8. Chat interface
def start_multimodal_chat():
    clip_model, clip_preprocess = init_clip()

    corpus = [
        "The sun is a star at the center of the Solar System.",
        "Cats are small, carnivorous mammals that are often kept as pets.",
        "The Eiffel Tower is a famous landmark in Paris, France."
    ]

    qa_chain = create_multimodal_qa(corpus)
    clip_text_model = SentenceTransformer("clip-ViT-B-32")
    corpus_embeddings = clip_text_model.encode(corpus)
    index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
    index.add(corpus_embeddings)

    print("\n🧠 Multimedia RAG Ready! Type 'exit' to quit.\n")

    while True:
        mode = input("Choose input mode (text/image/audio): ").strip().lower()
        if mode == "exit":
            print("👋 Exiting.")
            break
        elif mode == "text":
            query = input("📝 Your question: ")
        elif mode == "image":
            path = input("📁 Enter image path: ")
            try:
                image_features = embed_image(path, clip_model, clip_preprocess)
                D, I = index.search(image_features, k=1)
                query = corpus[I[0][0]]
                print(f"🧠 Interpreted image as: {query}")
            except Exception as e:
                print(f"⚠️ Error loading image: {e}")
                continue
        elif mode == "audio":
            path = input("🎵 Enter audio file path: ")
            try:
                query = transcribe_audio(path)
                print(f"🧠 Transcribed audio: {query}")
            except Exception as e:
                print(f"⚠️ Error transcribing audio: {e}")
                continue
        else:
            print("❌ Invalid mode. Please choose text, image, or audio.")
            continue

        try:
            result = qa_chain.invoke({"query": query})
            print(f"\n🤖 Answer: {result['result']}\n")
        except Exception as e:
            print(f"⚠️ Error during QA: {e}")

# Run
if __name__ == "__main__":
    start_multimodal_chat()


⚙️ Loading CLIP model...
📚 Embedding text chunks...
🧠 Loading FLAN-T5-Base...


Device set to use cpu


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

C:\Users\nares\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nares\.cache\huggingface\hub\models--sentence-transformers--clip-ViT-B-32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

🖼️ Embedding image using CLIP...
🧠 Interpreted image as: The Eiffel Tower is a famous landmark in Paris, France.

🤖 Answer: Cats are small, carnivorous mammals that are often kept as pets.



Choose input mode (text/image/audio):  E:\CV_LAB\Inputs\dog.jpg


❌ Invalid mode. Please choose text, image, or audio.


Choose input mode (text/image/audio):  exit


👋 Exiting.
